# IAU-PROJEKT, STAHOVEC a VIEST

------------------------------------------------
Projekt začína základnými importmi pre knižnice, ktoré budeme v projekte používať.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import mannwhitneyu
import statsmodels.api as sm
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

pd.options.mode.chained_assignment = None

In [ ]:
sns.set_theme(style="whitegrid", palette="viridis")
sns.set_palette("viridis")

Hneď na začiatok si **načítame dáta** zo súboru funkciou **read_csv()**
Následne vykonáme prvú "čistiacu" operáciu, v ktorej vymažeme prvý stĺpec, v ktorom
je očíslovanie stĺpcov, ktoré je pre našu analýzu zbytočné.

In [ ]:
data_labor = pd.read_csv("dataset/labor.csv", sep="\t")
# del data["Unnamed: 0"]
data_labor = data_labor.iloc[:, 1:]
data_labor

V nasledujúcich bunkách sme **nahradili chýbajúce hodnoty** priemerom v danom stĺpci.
Túto operáciu sme mohli vykonať z dôvodu, že všetky **chýbajúce hodnoty** boli iba v tých
stĺpcoch, ktoré obsahujú **numerické hodnoty** (float64).

Ešte predtým sme overili, či data_labor skutočne obsahuje nejaké chýbajúce hodnoty.

In [ ]:
data_labor.isna().sum()

In [ ]:
data_labor = data_labor.fillna(data_labor.mean())
data_labor

In [ ]:
data_labor.describe()

# 1. fáza - Prieskumná analýza

#### Percentuálny podiel práce: Stahovec = 50%, Viest = 50%

## a) Základný opis dát spolu s ich charakteristikami
-------------------------------------------------------
Základné informácie o datasete:

In [ ]:
print(f"Počet záznamov v našom datasete: {len(data_labor.index)}")
print(f"Počet atribútov v našom datasete: {len(data_labor.columns)}")
# print(f"Dátové typy v našom datasete:\n {data_labor.dtypes}")
print(f"Dátové typy v našom datasete: {list(set(data_labor.dtypes))}")

V tejto časti uvedieme pre zvolené **významné atribúty ich distribúcie a základné
deskriptívne štatistiky.**
Medzi tieto významné atribúty patria **erytrocyty, leukocyty, trombocyty, váha a
hemoglobín**.
Pre ich deskriptívne štatistiky uvádzame **medián, modus, priemer, rozptyl a smerodajnú odchylku.**
<br>

Hodnota modusu je väčšinou **rovná priemeru**, keďže nahradzujeme chýbajúce hodnoty priemerom. Je
potrebné brať tieto hodnoty s rezervou, keďže dáta ešte neboli očistené a mnoho záznamov spadá
do surreálnych hodnôt (napr. záporné záznamy pre atribút váhy).
</br>

###  Erytrocyty

Pri erytrocytoch vidíme, že na začiatku krivky je mierna odchýlka, ktorá spôsobuje to,
že táto distribúcia
nie je Gaussova aj keď, približne od hodnoty 5 na x-ovej osi by sa dalo povedať,
že distribúcia je normálová.

In [ ]:
sns.displot(data_labor["erytrocyty"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="erytrocyty", bins=24, kde=True)
print(f"[erytrocyty] Medián:  ", data_labor["erytrocyty"].median())
print(f"[erytrocyty] Modus:   ", data_labor.mode()["erytrocyty"][0])
print(f"[erytrocyty] Priemer: ", data_labor["erytrocyty"].mean())
print(f"[erytrocyty] Rozptyl: ", np.var(data_labor["erytrocyty"]))
print(f"[erytrocyty] Smerodajná odchylka: ", np.std(data_labor["erytrocyty"]))

### Leukocyty

Pri leukocytoch vidíme menší spike presne v strede distribúcie, čo môže byť náhoda
znásobená faktom, že sme nahradzovali chýbajúce hodnoty údajom, ktorý sa nachádza
práve na tom mieste.

In [ ]:
sns.displot(data_labor["leukocyty"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="leukocyty", bins=24, kde=True)
print(f"[leukocyty] Medián:  ", data_labor["leukocyty"].median())
print(f"[leukocyty] Modus:   ", data_labor.mode()["leukocyty"][0])
print(f"[leukocyty] Priemer: ", data_labor["leukocyty"].mean())
print(f"[leukocyty] Rozptyl: ", np.var(data_labor["leukocyty"]))
print(f"[leukocyty] Smerodajná odchylka: ", np.std(data_labor["leukocyty"]))


### Trombocyty

Pri trombocytoch vidíme pomerne veľkú odchýlku v strede distribúcie. Inak je distribúcia Gaussova,
čo znamená, že hodnoty sú rovnomerne rozložené naprieč grafom.

In [ ]:
sns.displot(data_labor["trombocyty"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="trombocyty", bins=24, kde=True)
print(f"[trombocyty] Medián:  ", data_labor["trombocyty"].median())
print(f"[trombocyty] Modus:   ", data_labor.mode()["trombocyty"][0])
print(f"[trombocyty] Priemer: ", data_labor["trombocyty"].mean())
print(f"[trombocyty] Rozptyl: ", np.var(data_labor["trombocyty"]))
print(f"[trombocyty] Smerodajná odchylka: ", np.std(data_labor["trombocyty"]))

### Váha

Ako sme vyčítali z metódy describe() na začiatku, tak na grafe sa nám potvrdilo,
že niektoré hodnoty váh siahajú do záporného regiónu. O to sa postaráme neskôr
v tomto notebooku.

In [ ]:
sns.displot(data_labor["weight"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="weight", bins=24, kde=True)
print(f"[weight] Medián:  ", data_labor["weight"].median())
print(f"[weight] Modus:   ", data_labor.mode()["weight"][0])
print(f"[weight] Priemer: ", data_labor["weight"].mean())
print(f"[weight] Rozptyl: ", np.var(data_labor["weight"]))
print(f"[weight] Smerodajná odchylka: ", np.std(data_labor["weight"]))

### Hemoglobin

Pri hemoglobíne vidíme, že distribúcia nie je Gaussova, pretože má veľký sklon a v maximálnych hodnotách
je krivka široká, to znamená, že veľká časť dát má hodnotu blízku k maximálnej.

In [ ]:
sns.displot(data_labor["hemoglobin"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="trombocyty", bins=24, kde=True)
print(f"[hemoglobin] Medián:  ", data_labor["hemoglobin"].median())
print(f"[hemoglobin] Modus:   ", data_labor.mode()["hemoglobin"][0])
print(f"[hemoglobin] Priemer: ", data_labor["hemoglobin"].mean())
print(f"[hemoglobin] Rozptyl: ", np.var(data_labor["hemoglobin"]))
print(f"[hemoglobin] Smerodajná odchylka: ", np.std(data_labor["hemoglobin"]))

## b) Párová analýza dát
-------------------------------------------------------
### **Skúmanie vzťahov medzi dvojicami atribútov**

<br>

V tejto časti zisťujeme **závislosti** medzi rôznymi atribútmi po dvojiciach. Keďže naše dáta
ešte stále nie sú očistené, výsledné závislosti **nemôžu byť stopercentne korektné**.
</br>
<br>
Na začiatok sme vykreslili **heatmapu**, z ktorej môžeme vyčítať **závislosti medzi všetkými dvojicami
atribútov**. Vďaka tomu máme všeobecný prehľad o koreláciách v našom datasete.
</br>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
heatmap = sns.heatmap(data_labor.corr(), linewidths=.5, square=True,
                      annot=True, linewidth=0.5, cmap="magma", fmt=".2f")
heatmap.set_title('Correlation Heatmap', pad=12);

Heatmapa z prechádzajúcej bunky nám napovedá, ktoré atribúty môžeme skúmať z hľadiska závislostí.
Na začiatok si vyberieme dvojice, ktoré majú vyšší koeficient korelácie, a teda tie
dvojice atribútov od seba závisia.
<br>

Korelácie medzi dvojicami, ktoré sme si vybrali na párovú analýzu:
- erytrocyty - alt
- hemoglobín - alp
- hemoglobín - etytr
- erytrocyty - leukocyty

Dvojice so závislou premennou:
- indikátor - erytrocyty
- indikátor - hematokrit
- indikátor - trombocyty
- indikátor - etytr


### Erytrocyty a alt
Na tomto grafe môžeme pozerať **silnú závislosť**, keďže rozptyl dát je exponenciálna krivka.
Pri atribútoch, ktoré nekorelujú, budeme vidieť rôzne atypické "machule".
</br>

In [ ]:
sns.scatterplot(data=data_labor, x="erytrocyty", y="alt")

### Hemoglobín a alp
Na danom grafe pozorujeme **sínusovú závislosť**, keďže osciluje v pravidelnom intervale.
V približnom odhade môžeme tvrdiť, že hodnoty atribútu alp dosahujú maximum vtedy,
ak sa hodnota hemoglobínu blíži k 6% a periodicky osciluje po približne šiestich
percentách.

In [ ]:
sns.scatterplot(data=data_labor, x="hemoglobin", y="alp")

### Hemoglobin a etytr
V danom porovnaní môžeme pozorovať miernu pozitívnu koreláciu medzi zvolenými atribútmi,
keďže distribúcia v grafe je, laicky povedané, jemne naklonená doprava hore.

In [ ]:
plt.subplots(figsize=(6, 6))
sns.scatterplot(data=data_labor, x="hemoglobin", y="etytr", s=5, color="#5e3c99")
sns.histplot(data=data_labor, x="hemoglobin", y="etytr", bins=50, pthresh=.1, cmap="viridis", shrink=.4)
sns.kdeplot(data=data_labor, x="hemoglobin", y="etytr", levels=8, color="#3b6cce")

Po relatívne závislých atribútoch sme prešli k prípadom, kedy sa korelácie pohybujú v záporných
hodnotách.

### Erytrocyty a Leukocyty
Ako prvú máme vykreslenú závislosť medzi atribútmi erytrocyty a leukocyty, ktorých hodnota korelácie
sa blíži v rádoch stotín k nule.

Nulová korelácia je pozorovateľná na grafe, keďže výskyt závislosti je roztrúsený v kruhovom tvare
uprostred grafu.

In [ ]:
# sns.scatterplot(data=data_labor, x="erytrocyty", y="leukocyty")

plt.subplots(figsize=(6, 6))
sns.scatterplot(data=data_labor, x="erytrocyty", y="leukocyty", s=5, color="#5e3c99")
sns.histplot(data=data_labor, x="erytrocyty", y="leukocyty", bins=50, pthresh=.1, cmap="viridis", shrink=.4)
sns.kdeplot(data=data_labor, x="erytrocyty", y="leukocyty", levels=8, color="#3b6cce")

### Erytrocyty a etytr
Pri porovnaní atribútov **erytrocyty** a **etytr** pozorujeme stredne veľkú negatívnu koreláciu,
keďže rozptyl dát je "negatívne naklonený", čo znamená, že pri rastúcej hodnote erytrocytov
klesá hodnota atribútu etytr.

In [ ]:
plt.subplots(figsize=(6, 6))
sns.scatterplot(data=data_labor, x="erytrocyty", y="etytr", s=5, color="#5e3c99")
sns.histplot(data=data_labor, x="erytrocyty", y="etytr", bins=50, pthresh=.1, cmap="viridis", shrink=.4)
sns.kdeplot(data=data_labor, x="erytrocyty", y="etytr", levels=8, color="#3b6cce")

### **Skúmanie závislostí medzi predikovanou premennou a ďalšími atribútmi**
V tejto časti sa povenujeme **závislostiam medzi predikovanou premennou** a ostatnými premennými.
Predikovaná premenná je v našom prípade **indicator**.

### Indikátor a erytrocyty (0.36)

Z heatmapy vieme vyčítať, že dvojica atribútov indikátor a erytrocyty je **mierne pozitívne závislá**.
Tieto rozdiely sú badateľné na grafe nižšie, keďže zhluky dát sú zľahka rozdielne deformované a
vertikálne posunuté.

In [ ]:
with sns.axes_style('white'):
    graph = sns.jointplot(x="indicator", y="erytrocyty", data=data_labor, kind='hex')
    graph.plot_joint(sns.kdeplot, color="#5e5ce9")
    graph.plot_marginals(sns.rugplot, color="blue", height=-.12, clip_on=False)

In [ ]:
sns.violinplot(data=data_labor, x='indicator', y = 'erytrocyty')

### Indikátor a hematokrit (0.69)

Na tejto dvojici atribútov pozorujeme veľmi **silnú pozitívnu závislosť**, ktorá je viditeľná na oboch
typoch grafov (violin plote aj joint plote). S nárastom hodnoty indikátoru z 0 na 1 výrazne rastie
aj hodnota hematokritu.

In [ ]:
with sns.axes_style('white'):
    graph = sns.jointplot(x="indicator", y="hematokrit", data=data_labor, kind='hex')
    graph.plot_joint(sns.kdeplot, color="#5e5ce9")
    graph.plot_marginals(sns.rugplot, color="blue", height=-.12, clip_on=False)

In [ ]:
sns.violinplot(data=data_labor, x='indicator', y = 'hematokrit')


### Indikátor a trombocyty (0.027)

**Takmer nulová závislosť** - hodnoty hematokritu sú takmer zhodné pre rôzne hodnoty indikátoru.

In [ ]:
with sns.axes_style('white'):
    graph = sns.jointplot(x="indicator", y="trombocyty", data=data_labor, kind='hex')
    graph.plot_joint(sns.kdeplot, color="#5e5ce9")
    graph.plot_marginals(sns.rugplot, color="blue", height=-.12, clip_on=False)

In [ ]:
sns.violinplot(data=data_labor, x='indicator', y = 'trombocyty')

### Indikátor a etytr (-0.38)

**Mierne negatívna závislosť** je viditeľná tak, že pri náraste hodnoty indikátoru klesá
hodnota atribútu etytr. Distribúcia pri rôznom indikátore je taktiež **odlišná**.

In [ ]:
with sns.axes_style('white'):
    graph = sns.jointplot(x="indicator", y="etytr", data=data_labor, kind='hex')
    graph.plot_joint(sns.kdeplot, color="#5e5ce9")
    graph.plot_marginals(sns.rugplot, color="blue", height=-.12, clip_on=False)

In [ ]:
sns.violinplot(data=data_labor, x='indicator', y = 'etytr')

## c) Formulácia a štatistické overenie hypotéz o dátach
-------------------------------------------------------
### Formulácia hypotéz

#### 1. Hypotéza:
##### (**H0**): Pacienti v lepšom stave majú rovnaké hodnoty erytrocytov v krvi ako pacienti
##### v horšom stave.
##### (**H1**): Pacienti v lepšom stave majú iné hodnoty erytrocytov v krvi ako pacienti
##### v horšom stave.

#### 2. Hypotéza:
##### (**H0**): Pacienti v horšom stave majú hodnoty objemu krviniek (hematokrit) rovnaké
##### ako pacienti v lepšom stave.
##### (**H1**): Pacienti v horšom stave majú hodnoty objemu krviniek (hematokrit) rôzne
##### ako pacienti v lepšom stave
-------------------------------------------------------

Tieto hypotézy sme si zvolili z toho dôvodu, že predchádzajúca časť nám naznačila tieto poznatky,
z ktorých vychádzajú dané hypotézy. Pred samotným začiatkom štatistického overenia hypotéz
vykreslíme zopár grafov, z ktorých zistíme, či sú dané **dáta vhodné** na použitie štatistických
testov.

#### 1. Hypotéza:

V prvom boxenplote môžeme vidieť iný typ porovnania závislostí indikátora od erytrocytov, ako
sme použili v predchádzajúcej časti, no výsledok je v podstate rovnaký.
Pozorujeme miernu koreláciu medzi zvolenou dvojicou atribútov.

In [ ]:
g = sns.boxenplot(data=data_labor, x='indicator', y = 'erytrocyty')

Následne skontrolujeme atribúty indikátor a erytrocyty a ich hodnoty, napr. či sú ich distribúcie
normálne alebo či ich dáta nezasahujú do záporných hodnôt, čo je z logického hľadiska nezmysel.

In [ ]:
data_labor[['erytrocyty', 'indicator']].describe()

In [ ]:
sns.displot(data_labor["erytrocyty"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="erytrocyty", bins=24, kde=True)
print(f"[erytrocyty] Medián:  ", data_labor["erytrocyty"].median())
print(f"[erytrocyty] Modus:   ", data_labor.mode()["erytrocyty"][0])
print(f"[erytrocyty] Priemer: ", data_labor["erytrocyty"].mean())
print(f"[erytrocyty] Rozptyl: ", np.var(data_labor["erytrocyty"]))
print(f"[erytrocyty] Smerodajná odchylka: ", np.std(data_labor["erytrocyty"]))

In [ ]:
sns.displot(data_labor["indicator"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="erytrocyty", bins=24, kde=True)
print(f"[indicator] Medián:  ", data_labor["indicator"].median())
print(f"[indicator] Modus:   ", data_labor.mode()["indicator"][0])
print(f"[indicator] Priemer: ", data_labor["indicator"].mean())
print(f"[indicator] Rozptyl: ", np.var(data_labor["indicator"]))
print(f"[indicator] Smerodajná odchylka: ", np.std(data_labor["indicator"]))

Z danej krátkej analýzy dvoch atribútov sme zistili, že hodnoty indikátora sú
v očakávanej norme a hodnoty erytrocytov sú pre danú analýzu taktiež
akceptovateľné.

Následne sme rozdelili dataset do **dvoch podsetov podľa hodnoty indikátora**,
aby sme mohli pracovať s našimi hypotézami.

#### Delenie atribútov do dvoch setov podľa indikátora:

In [ ]:
data_0_erytrocyty = data_labor.loc[data_labor["indicator"] == 0, "erytrocyty"]
data_0_erytrocyty

In [ ]:
sns.displot(data_0_erytrocyty, kde=True, bins=32, label="Indicator 0")
plt.legend()

In [ ]:
sm.qqplot(data_0_erytrocyty, fit=True, line="45")
plt.show()

In [ ]:
data_1_erytrocyty = data_labor.loc[data_labor["indicator"] == 1, "erytrocyty"]
data_1_erytrocyty

In [ ]:
sns.displot(data_1_erytrocyty, kde=True, bins=32, label="Indicator 1")
plt.legend()

In [ ]:
sm.qqplot(data_1_erytrocyty, fit=True, line="45")
plt.show()


Aby sme dodržali korektnosť pri vykonávaní nasledujúcich testov, rozhodli sme sa
z našich datasetov vybrať rovnako veľkú vzorku pre oba hodnoty indikátoru. Tým zaručíme
aj použiteľnosť Shapiro-Wilkovho testu, ktorý je vhodný pre dáta do 5000 záznamov.

In [ ]:
print(data_0_erytrocyty.count())
print(data_1_erytrocyty.count())

Kedže počet záznamov v dataframe data_0_erytrocyty **(3558)** je menší ako počet záznamov v
dataframe data_1_erytrocyty **(6413)**, z data_1_erytrocyty vyberieme **toľko záznamov, koľko je v
data_0_erytrocyty**

In [ ]:
data_1_erytrocyty = data_1_erytrocyty.sample(data_0_erytrocyty.count())
print(data_0_erytrocyty.count())
print(data_1_erytrocyty.count())

Z predchádzajúcich grafov je nám naznačené, že **nepôjde o normálnu distribúciu**.
Následne sme si vypočítali Cohenovo déčko, aby sme vedeli určiť silu nášho testu,
resp. silu presvedčenia o výsledku, ktorý dostaneme v ďalších testoch.

Výsledok je väčší ako učebnicová hodnota 0.75, čo znamená, že výsledok nášho testu
bude dostatočne presvedčivý.

In [ ]:
def cohen_d(x1, x2):
    nx1 = len(x1)
    nx2 = len(x2)
    s = np.sqrt(((nx1-1) * np.std(x1, ddof=1)**2 + (nx2-1) * np.std(x2, ddof=1)**2) / (nx1 + nx2 - 2))
    return (np.abs(np.mean(x1) - np.mean(x2))) / s

c_d_erytrocyty = cohen_d(data_0_erytrocyty, data_1_erytrocyty)
c_d_erytrocyty


Keďže sme si dáta rozdelili do dvoch setov podľa hypotézy, prešli sme na
**overenie nulovej hypotézy**, aby sme sa uistili,
že dáta pochádzajú z normálnej distribúcie.
Predpokladali sme, že dáta budú pochádzať skôr z rôznych distribúcií ako normálnych.

Na to nám slúži **Shapiro-Wilkov test**.

In [ ]:
sns.histplot(data=data_0_erytrocyty)
alpha = 0.05
shapiro_test = stats.shapiro(data_0_erytrocyty)
print(shapiro_test.pvalue)

if shapiro_test.pvalue > alpha:
    print('Normálna distribúcia (fail to reject H0)')
else:
    print('Iná distribúcia (reject H0)')

In [ ]:
sns.histplot(data=data_1_erytrocyty)
alpha = 0.05
shapiro_test = stats.shapiro(data_1_erytrocyty)
print(shapiro_test.pvalue)

if shapiro_test.pvalue > alpha:
    print('Normálna distribúcia (fail to reject H0)')
else:
    print('Iná distribúcia (reject H0)')

Keďže podľa Shapiro-Wilkovho testu distribúcia oboch vzoriek **nepatrí
do skupiny normálnych** a naše dáta sú nepárové, pokračujeme testom, ktorý sa
nazýva **Mann-Whitneyho test**. Mann-Whitneyho test použijeme ako potvrdenie
výsledku SW testu.

In [ ]:
alpha = 0.05
stat, p = mannwhitneyu(data_0_erytrocyty, data_1_erytrocyty)
print(p)

if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

#### Záver 1. hypotézy

Mann-Whitneyho test potvrdil, že hypotézu **H0 môžeme zamietnuť**
s dostatočnou mierou presvedčenia (aj kvôli Cohen's d), vďaka čomu
môžeme tvrdiť, že pacienti v lepšom stave majú **nie rovnaké hodnoty erytrocytov**
v krvi. Minimálne výsledné hodnoty oboch testov naznačujú fakt, že hypotéza
H0 nie je konzistentná.


#### 2. Hypotéza:

##### (**H0**): Pacienti v horšom stave majú hodnoty objemu krviniek (hematokrit) rovnaké
##### ako pacienti v lepšom stave.
##### (**H1**): Pacienti v horšom stave majú hodnoty objemu krviniek (hematokrit) rôzne
##### ako pacienti v lepšom stave.
-------------------------------------------------------

Opäť sme začali iným typom zobrazenia závislostí medzi zvolenými atribútmi (tentokrát
šlo o indikátor a hematokrit).

In [ ]:
g = sns.boxenplot(data=data_labor, x='indicator', y = 'hematokrit')

Následne skontrolujeme atribúty indikátor a hematokrit a ich hodnoty, napr. či sú ich distribúcie
normálne alebo či ich dáta nezasahujú do záporných hodnôt, čo je z logického hľadiska nezmysel.

In [ ]:
data_labor[['hematokrit', 'indicator']].describe()

In [ ]:
sns.displot(data_labor["hematokrit"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="erytrocyty", bins=24, kde=True)
print(f"[hematokrit] Medián:  ", data_labor["hematokrit"].median())
print(f"[hematokrit] Modus:   ", data_labor.mode()["hematokrit"][0])
print(f"[hematokrit] Priemer: ", data_labor["hematokrit"].mean())
print(f"[hematokrit] Rozptyl: ", np.var(data_labor["hematokrit"]))
print(f"[hematokrit] Smerodajná odchylka: ", np.std(data_labor["hematokrit"]))

In [ ]:
sns.displot(data_labor["indicator"], kde=True, bins=32)
# sns.histplot(data=data_labor, x="erytrocyty", bins=24, kde=True)
print(f"[indicator] Medián:  ", data_labor["indicator"].median())
print(f"[indicator] Modus:   ", data_labor.mode()["indicator"][0])
print(f"[indicator] Priemer: ", data_labor["indicator"].mean())
print(f"[indicator] Rozptyl: ", np.var(data_labor["indicator"]))
print(f"[indicator] Smerodajná odchylka: ", np.std(data_labor["indicator"]))

Z tejto krátkej štatistickej analýzy sme zistili, že hodnoty oboch atribútov
sa pohybujú v očakávaných intervaloch, no distribúcia hematokritu nám naznačila
možný výsledok Shapiro-Wilkovho testu.

Následne sme si rozdelili dataset do **dvoch podsetov podľa hodnoty indikátora**.

In [ ]:
data_0_hematokrit = data_labor.loc[data_labor["indicator"] == 0, "hematokrit"]
data_0_hematokrit

In [ ]:
sns.displot(data_0_hematokrit, kde=True, bins=32, label="Indicator 0")
plt.legend()

In [ ]:
sm.qqplot(data_0_hematokrit, fit=True, line="45")
plt.show()

In [ ]:
data_1_hematokrit = data_labor.loc[data_labor["indicator"] == 1, "hematokrit"]
data_1_hematokrit

In [ ]:
sns.displot(data_1_hematokrit, kde=True, bins=32, label="Indicator 1")
plt.legend()

In [ ]:
sm.qqplot(data_1_hematokrit, fit=True, line="45")
plt.show()

Z daných dát je očividná **rozdielnosť** týchto dvoch vzoriek ako aj ich **distribúcia**,
ktorá má od normálnosti pomerne ďaleko.

Prešli sme k časti vybratia rovnako veľkej vzorky pre rôzne hodnoty predikovanej premennej.
Dôvodom bolo opäť, zaručenie rovnakých podmienok
pre nasledovné štatistické testy.

In [ ]:
print(data_0_hematokrit.count())
print(data_1_hematokrit.count())

V tomto prípade rovnako ako v prípade erytrocytov je počet záznamov pre data_0_hematokrit **(3558)**
nižší ako počet záznamov pre data_1_hematokrit **(6413)**, takže sme tiež vybrali z data_1_hematokrit
rovanký počet záznamov ako pre data_0_hematokrit.

In [ ]:
data_1_hematokrit = data_1_hematokrit.sample(data_0_hematokrit.count())
data_1_hematokrit

In [ ]:
print(data_0_hematokrit.count())
print(data_1_hematokrit.count())

Z predchádzajúcich grafov je jasné, že **nepôjde o normálnu distribúciu**.
Následne sme prešli na výpočet **Cohen's d**.

In [ ]:
c_d_hematokrit = cohen_d(data_0_hematokrit, data_1_hematokrit)
c_d_hematokrit

Výsledok je väčší ako učebnicová hodnota 1 a pohybuje sa okolo 2,
čo znamená, že **rozdiel medzi priemermi našich dvoch datasetov je rádovo
väčší ako 2 štandardné odchýlky**.

Vďaka tejto informácií môžeme tvrdiť, že výsledok nasledujúcich testov je
**veľmi dôveryhodný**.

Shapiro-Wilkov test overíme aj napriek qq-plotom,
aby sme si mohli byť istí, že nejde o normálnu distribúciu.

In [ ]:
sns.histplot(data=data_0_hematokrit)
alpha = 0.05
shapiro_test = stats.shapiro(data_0_hematokrit)
print(shapiro_test.pvalue)

if shapiro_test.pvalue > alpha:
    print('Normálna distribúcia (fail to reject H0)')
else:
    print('Iná distribúcia (reject H0)')

In [ ]:
sns.histplot(data=data_1_hematokrit)
alpha = 0.05
shapiro_test = stats.shapiro(data_1_hematokrit)
print(shapiro_test.pvalue)

if shapiro_test.pvalue > alpha:
    print('Normálna distribúcia (fail to reject H0)')
else:
    print('Iná distribúcia (reject H0)')

Shapiro-Wilkov test nám vrátil **extrémne nízke p-hodnoty**, čo naznačuje vyvrátenie
nulovej hypotézy. Daný predpoklad sme si ešte potvrdili **Mann-Whitneyho testom**
v nasledujúcej bunke.

In [ ]:
alpha = 0.05
stat, p = mannwhitneyu(data_0_hematokrit, data_1_hematokrit)
print(p)

if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

#### Záver 2. hypotézy

Mann-Whitneyho test potvrdil, že hypotézu **H0 môžeme zamietnuť** v prospech H1.
To znamená, že môžeme s dostatočným presvedčením predpokladať znenie alternatívnej
hypotézy.

## d) Identifikácia problémov v dátach s navrhnutým riešením
-------------------------------------------------------
### Identifikácia problémov

Táto sekcia prvej fázy je venovaná **čisteniu dát**, respektíve **hľadaniu problémov**
v datasete s návrhom riešení pre dané problémy. Všetky navrhnuté zmeny sme zakomentovali,
aby sme mali funkčný k dispozícii už do druhej fázy.

Postupovali sme problém po probléme v oboch .csv súboroch naraz.

### Nevhodná štruktúra dát

Na úvod sme otvorili druhý .csv súbor s názvom profiles.csv a vykonali sme základné
úpravy ako aj pri labor.csv na začiatku notebooku (vyhodenie prvého stĺpca, kde sú len
čísla).

In [ ]:
data_profiles = pd.read_csv("dataset/profiles.csv", sep="\t")
data_profiles = data_profiles.iloc[:, 1:]
data_profiles

Pred samotnou úpravou nevhodných štruktúr dát sme si vypísali prvých 20 záznamov,
na ktorých sme sa pokúsili detekovať abnormality v ich štruktúre zápisu.
Dataset data_labor bol z tohto ohľadu v poriadku.

In [ ]:
data_labor.head(20)

In [ ]:
data_profiles.head(20)

V data_profiles sme si všimli hneď niekoľko problémov v našich záznamoch. Prvý z nich
bol hneď v atribúte birthdate, kde bol dátum narodenia zapísaný niekoľkými rôznymi
spôsobmi.

Ďalšia nezrovnalosť bola v atribútoch address a residence, kde sa v záznamoch nachádzali
znaky pre nový riadok a ďalšie formátovacie znaky typu \r apod.

Poslednou chybou v rámci pozorovania prirodzeného jazyka boli záznamy pre current_location,
ktoré mali nevhodný formát.

Naše navrhnuté riešenia sú zakomentované nižšie.

* Pre atribút birthdate navrhujeme použitie predefinovaného pandas formátu, ktorý
by v takomto prípade nahradil všetky rozličné formáty jednotným.
* Pre atribúty currect_location, address a residence možno použiť nahradzovanie
regulárnym výrazom a vstavanou metódou replace(), ktorou by sa nahradili všetky
nechcené znaky, ktoré sa v záznamoch nachádzajú.

In [ ]:
# data_profiles[["birthdate"]] = data_profiles[["birthdate"]].apply(pd.to_datetime)

In [ ]:
# data_profiles["current_location"] = data_profiles["current_location"].str.replace(r"[A-Za-z]", "", regex=True)
# data_profiles["current_location"] = data_profiles["current_location"].str.replace(r"[()']", "", regex=True)
# data_profiles

In [ ]:
# data_profiles["address"] = data_profiles["address"].str.replace(r"[\r\n]", " ", regex = True)
# data_profiles

In [ ]:
# data_profiles["residence"] = data_profiles["residence"].str.replace(r"[\r\n]", " ", regex = True)
# data_profiles

### Duplicitné záznamy

Ďalším problémom v datasete, s ktorým sa je potrebné popasovať, je riešenie **duplikátnych riadkov**.
To je z hľadiska trénovanie umelej inteligencie podstatné, keďže **duplikátny riadok nepridáva
nijakú hodnotu** pre náš algoritmus, skôr môže narušiť jeho úsudok v prospech týchto rovnakých
riadkov.

V tejto bunke sme spočítali riadky, ktoré majú **duplikát** v celom datasete.

In [ ]:
data_labor.duplicated(keep=False).sum()

Pre tento problém by sme zvolili riešenie, v ktorom by sme odstránili všetky duplikátne
riadky s tým, že by bol ponechaný len jeden z nich. V kóde, ktorý je zakomentovaný nižšie,
je aj kontrola, či sa daný úkon vykonal správne. Po takejto úprave je potrebné preindexovať
záznamy nanovo.

In [ ]:
# data_labor.drop_duplicates(inplace=True)
# data_labor.duplicated(keep=False).sum()
# data_labor.index = range(len(data_labor))
# data_labor

Naša datábaza profilov neobsahuje duplikátne riadky, čo je overené v nasledujúcej bunke.
Preto nie je potrebné v tomto sete riešiť nijakú duplicitu.

In [ ]:
data_profiles.duplicated(keep=False).sum()

### Nejednotné formáty

Do okruhu nejednotných formátov spadajú prípady ako napr. rôzna veľkosť písmen,
preklepy, rôzne podoby zápisov alebo aj ich kombinácia.

Preto sme si na úvod vypísali všetky unikátne v hodnoty v atribútoch, pre
ktoré dáva zmysel riešiť nejednotný formát
(napr. pri atribúte rasy to dáva zmysel, keďže rás je len zopár unikátnych,
kdežto adresy takto kontrolovať nemôžeme).

Atribúty v data_labor, pri ktorých možno kontrolovať jednotný formát:
* smoker
* relationship

Atribúty v data_profiles, pri ktorých možno kontrolovať jednotný formát:
* race
* blood_group
* sex

### Nejednotný formát: data_labor

In [ ]:
data_labor["smoker"].unique()

V datasete data_labor pre atribút smoker sme identifikovali nejednotný formát,
v 4 rôznych verziách. Na tento problém by sme vykonali prekonvertovanie hodnôt
na numerické hodnoty 0 a 1, keďže to sú jediné možnosti, ako môže byť daný
atribút vyjadrený.

In [ ]:
"""
data_labor["smoker"] = data_labor["smoker"].str.replace("yes", "1")
data_labor["smoker"] = data_labor["smoker"].str.replace("no", "0")
data_labor["smoker"] = data_labor["smoker"].str.replace("Y", "1")
data_labor["smoker"] = data_labor["smoker"].str.replace("N", "0")
data_labor["smoker"] = data_labor["smoker"].astype(float)
"""

In [ ]:
data_labor["relationship"].unique()

Atribút relationship bol naformátovaný relatívne korektne, no upravili by sme
hrúbku pre slovo "divoced" na "divorced" pre lepšiu zrozumiteľnosť záznamov.

In [ ]:
# data_labor["relationship"] = data_labor["relationship"].str.replace("divoced", "divorced")

### Nejednotný formát: data_profiles

In [ ]:
data_profiles["race"].unique()

Pre rasy sme identifikovali preklep v prípade rasy **black**, kde bolo **blsck**.
Tento preklep by sme upravili funkciou replace().
Taktiež sme si všimli, že sa jednotlivé rasy začínajú na malé aj veľké písmená.
Túto skutočnosť by sme upravili pomocou funkcie lower(), ktorá zmení všetky veľké
písmená na malé.

In [ ]:
# data_profiles["race"] = data_profiles["race"].str.replace("blsck", "black")
# data_profiles["race"] = data_profiles["race"].str.lower()

In [ ]:
data_profiles["blood_group"].unique()

Pre atribút blood_group boli formáty jednotné typu: veľké písmeno a +/-.
V budúcnosti možno pristúpime ku kroku nahradzovania týchto string hodnôt
na numerickú podobu.

In [ ]:
data_profiles["sex"].unique()

Tak ako pri atribúte smoker v predchádzajúcej časti, tak aj pri atribúte
pohlavia by sme vykonali zmenu formátu na numerické hodnoty 0 a 1.

In [ ]:
"""
data_profiles["sex"] = data_profiles["sex"].str.replace("M", "1")
data_profiles["sex"] = data_profiles["sex"].str.replace("F", "0")
data_profiles["sex"] = data_profiles["sex"].astype(float)
"""

Príkaz spustený v bunke nižšie kontroluje, či sú všetky hodnoty vo všetých stĺpcoch
**numerické**, vďaka čomu by sme mohli prehlásiť,
že dané atribúty obsahujú výlučne numerické hodnoty.

V kombinácii spolu s funkciou info() vieme potvrdiť, že ide o dátový typ float_64 a
že ostatné dátové typy sú typu object, čo je Python reprezentácia stringu.

In [ ]:
data_labor.apply(lambda y: pd.to_numeric(y, errors='coerce').notnull().all())

In [ ]:
data_labor.info()

In [ ]:
data_profiles.apply(lambda y: pd.to_numeric(y, errors='coerce').notnull().all())

In [ ]:
data_profiles.info()

### Chýbajúce hodnoty

Chýbajúce hodnoty sme na našom datasete sčasti už upravili.

Pre data_labor sme ešte pred začiatkom prvej fázy vykonali nahradenie priemerom.
Túto skutočnosť, že náš dataset už naozaj nemá nijaké chýbajúce hodnoty, možno
overiť bunkou nižšie, ktorá zráta počet chýbajúcich hodnôt pre každý atribút
zvlášť.

In [ ]:
data_labor.isna().sum()

Pre data_profiles navrhujeme rovnakú taktiku, ktorá v prvom rade spočíva v zistení,
či sa v datasete nachádzajú nejaké chýbajúce hodnoty. Ako možno vidieť v bunke nižšie,
tak chýbajúce dáta nemáme, a preto nijaké ďalšie kroky nie sú potrebné.

In [ ]:
data_profiles.isna().sum()

### Vychýlené hodnoty

V tejto sekcii sú navrhnutú postupy riešenia problémov s vychýlenými hodnotami.

Na začiatok je vhodné poznamenať, že problém vychýlených hodnôt sa týka len
číselných záznamov, a preto atribúty typu string nie je potrebné ošetrovať.
Začali sme výpisom datatypov v jednotlivých setoch.

In [ ]:
data_labor.info()

In [ ]:
data_profiles.info()

Zistili sme, že data_profiles obsahuje výlučne stringy, z tohto dôvodu nie je
potrebná korekcia pre vychýlené hodnoty.

Zoznam číselných atribútov, pre ktoré je možné upravovať tzv. outlierov:
* hbver
* erytrocyty
* alt
* leukocyty
* etytr
* hemoglobin
* trombocyty
* alp
* er-cv
* weight
* ast
* hematokrit

Na začiatok sme sa rozhodli pre výpis základných štatistkých údajov metódou
describe().

In [ ]:
data_labor.describe()

Z predchádzajúcej bunky sme zistili, že niektoré atribúty majú hodnoty viac
roztrúsené ako iné. Jedným z extrémov bola váha, ktorej hodnoty siahali až
do záporných hodnôt.

Skrz tieto zistenia sme sa rozhodli, že váhu budeme upravovať samostatne a všetky
zvyšné numerické atribúty ošetríme spoločne.

Pre atribút váhy by sme navrhli riešenie pripočítania priemeru k hodnotám, ktoré
spadajú pod istú hranicu s ohľadom na pohlavie.

Na úpravu outlierov pre ostatné atribúty navrhujeme využiť metódu detekcie extrémov,
ktoré ležia
za dolnou alebo hornou hranicou, ktoré možno vypočítať z tzv. IQR (Interquartile range).

### VÁHA

Náš návrh spočíva v zistení priemerných hodnôt u oboch pohlaví, čo by mohlo byť
zohľadnené pri nahradzovaní záporných hodnôt.

V bunkách nižšie je príklad kódu, ktorým by sme zistili rôzne hodnoty váh
u mužov a žien.

In [ ]:
# merged_data = pd.merge(data_labor, data_profiles, on="ssn")
# merged_data.head(15)

In [ ]:
# data_f_weight = merged_data.loc[merged_data["sex"] == "F", "weight"]
# data_f_weight.describe()

In [ ]:
# data_m_weight = merged_data.loc[merged_data["sex"] == "M", "weight"]
# data_m_weight.describe()

Nižšie sa nachádza vzorový kód, ktorý by mohol byť použiteľný v prípade, ak by
hodnoty priemerných váh u oboch pohlaví boli identické alebo aspoň podobné.

In [ ]:
"""
for x in data_labor.index:
    if data_labor.loc[x, "weight"] < 5:
        data_labor["weight"][x] = data_labor["weight"][x] + data_labor["weight"].mean()
"""


Všetky ostatné atribúty by sme riešili pomocou metódy quantile_calc().
V tejto metóde počítame IQR pomocou daného vzorca s tým, že hodnotu IQR
použijeme na určenie lower a upper whisker. Tieto hodnoty sú hraničné, pričom
ak sa im hodnoty atribútov vymykajú, ide o tzv. outliers,
ktorých hodnoty budú následne upravené.

Na záver funkcia vykreslí aj boxplot na vizuálne zobrazenie kvartilov a whiskerov
a vráti tieto hraničné hodnoty.

In [ ]:
def quantile_calc(column, boxplot=True):
    Q1 = data_labor[column].quantile(0.25)
    Q3 = data_labor[column].quantile(0.75)
    IQR = Q3 - Q1
    print(Q1)
    print(Q3)
    print(IQR)
    Lower_Whisker = Q1-1.5*IQR if Q1-1.5*IQR > 0 else 0
    Upper_Whisker = Q3+1.5*IQR
    print(Lower_Whisker, Upper_Whisker)

    if boxplot:
        sns.boxplot(x=data_labor[column])

    return Lower_Whisker, Upper_Whisker

Pomocou tejto funkcie a konkrétnych hraničných hodnôt pre jednotlivé atribúty
by sme upravili ich distribúcie individuálne.
Ak by daný atribút potreboval individuálny prístup v zmysle, že jeho vychýlené
hodnoty sú roztrúsené omnoho viac ako pri normálnejších distribúciách,
použili by sme ručne napísaný kód.
V iných prípadoch by sme použili funkciu move_outliers(), ktorej kód je nižšie.

In [ ]:
def move_outliers(lower, upper, attr):
    for i in data_labor.index:
        if data_labor.loc[i, attr] < lower:
            data_labor[attr][i] = data_labor[attr][i] + lower
        elif data_labor.loc[i, attr] > upper:
            data_labor[attr][i] = data_labor[attr][i] - lower

    sns.boxplot(x=data_labor[attr])

Príklady, ako by sme tieto úpravy vykonali, sú zobrazené nižšie aj s dostupným
kódom. Pri atribúte alt je ukážka, kedy bol potrebný individuálny prístup, keďže
sme chceli ponechať outlierov outliermi, ale v rozumnejšej miere, aby sa nenarušila
podstata vychýlených hodnôt, ktoré taktiež ukazujú dôležitú výpovednú hodnotu.

## ALT

In [ ]:
lower, upper = quantile_calc("alt")

Zistili sme, že máme priveľa **vychýlených hodnôt** a nemôžeme všetky vyhodiť z nášho datasetu.
Preto sme sa rozhodli pre metódu úpravy hodnôt pomocou **kvantilov** s individuálnym prístupom.

In [ ]:
# 2.5 -> 2.05
# 5 -> 2.10
# 20 -> 2.40
# 100 -> 4.00

for x in data_labor.index:
    if data_labor.loc[x, "alt"] > upper:
        data_labor["alt"][x] = (data_labor["alt"][x]/(upper*25)) + upper

sns.boxplot(x=data_labor["alt"])

## HEMOGLOBÍN

Ako úpravu sme opäť zvolili pripočítanie istej hodnoty k "outlierom", aby sme ich
"shiftli" do normálneho intervalu.

In [ ]:
lower, upper = quantile_calc("hemoglobin")

In [ ]:
move_outliers(lower, upper, "hemoglobin")

## ALP

Hodnoty alpu sme nemuseli upravovať, keďže boxplot z našej funkcie nevykreslil nijakých
outlierov. Preto sme sa rozhodli daný atribút neupravovať.

In [ ]:
quantile_calc("alp")

## ER-CV

Opäť raz sme použili našu funkciu **quantile_calc()**, ktorá nám okrem
hraničných hodnôt vykreslila boxplot, v ktorom môžeme graficky odpozorovať
vymykajúce sa hodnoty er-cv atribútu.

In [ ]:
lower, upper = quantile_calc("er-cv")

In [ ]:
move_outliers(lower, upper, "er-cv")

## AST

Pri atribúte AST sme zvolili identickú taktiku ako pri atribúte er-cv.
* výpočet hraníc a vykreslenie boxplotu
* úprava outlierov pripočítaním/odpočítaním spodnej hranice

In [ ]:
lower, upper = quantile_calc("ast")

In [ ]:
move_outliers(lower, upper, "ast")

In [ ]:
data_labor.describe()